# Semantic Networks

This example is about calculating cooccurences for words from a corpus and visualize the semantic network. We will use the Dice coefficient and mutual information.

The context of cooccurring words can vary from left or right neighbours and word windows (ngrams) to paragraphs and even documents. For this exercise, consider sentence cooccurrences. In this case document and sentence are often used interchangably (as in: it is still called Document-Term-Matrix). 

To that end, we first have to create the document-term-matrix our corpus and then calculate the Term-Term-Matrix from there. In practice these  are highly sparse and quickly face us with a memory issue. That is why it is important to consider sparse data structures.

`from scipy.sparse import coo_matrix, csr_matrix` provides sparse array representations. For further information see [here](https://docs.scipy.org/doc/scipy/reference/sparse.html).
Once established these can be (almost) handled like the usual numpy arrays, while keeping a low memory footprint.

Also python has some language paradigms that make some larger data settings somewhat involved. A good thing for example is to retreat to the use of generators instead of lists where ever large data needs to be transformed.
Vectorization is not native to python at this point, unfortunately.
See [here](https://realpython.com/list-comprehension-python/#when-not-to-use-a-list-comprehension-in-python).

Since this is very technical and very specific to a programing language, the part of creating the DTM is given in this exercise. (see method create_dtm). executing this you can access `vocab`, `dtm` and `cocounts`. 

After these initial thoughts the signifance of cooccurring words can be calculated.

# Word sense by Cooccurrence

#### Todo:
Use the  State of the union adresses uploaded in the moodle (or any corpus from a previous exercise or of your liking).<br>

    1) Write methods to calculate the DICE coefficient and the Mutual Information (MI).
    2) Write a function that returns the n-most significant cooccurrences of a word
  
  
To detect the semantic concepts and contexts behind a word, an easy way is to visualize the graph of secondary coocurrences. Secondary cooccurrences are the cooccurrences of coocurring words.

    3) Write a function that returns the n-most coocurrences and n-most secondary cooccurrences for every word.
       The format should be a list of (w1,w2) if w1 and w2 are cooccurrences.
       Result:    ```[(w1,w2), (w1,w3), (w1, w4), (w3,w4),....]```
       
    4) Use the function `visualize_semantic_network` to visualize some terms (e.g. California or President in sotu). Explain what you see.
    

In [ ]:
#!pip install --quiet tqdm networkx matplotlib scipy
import nltk
import numpy as np
from tqdm import tqdm
from scipy.sparse import coo_matrix, csr_matrix
import zipfile
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# You can use the topic corpora from the keyword extraction session:
content = {}
with zipfile.ZipFile("keyword.zip") as zfile:
    for f in zfile.namelist():
        if f != "keyword/":
            content[f] = zfile.read(f).decode("utf8")
content.keys()

In [ ]:
#Or a new corpus of State-of-the-Union adresses.
#with open("coocs/sotu.csv", "r") as f : sotu = f.read()

In [ ]:
class Coocs:
    def create_dtm(self, texts, cut_first=200, min_freq=5):
        
        # Clean texts
        self.texts = nltk.sent_tokenize(texts, language="german")[:-1]
        self.texts = map(lambda x: nltk.word_tokenize(x, language="german"), self.texts,)# for x in tqdm(self.texts)]
        
        ## count and restrict domain level text to the vocabulary
        self.term_frequencies = [nltk.FreqDist(text) for  text in tqdm(self.texts)]
        
        ## Count texts
        print("aggregating....")
        self.vocab = {}
        for e in tqdm(self.term_frequencies):
            for k, v in e.items():
                self.vocab[k] = self.vocab.get(k,0) + v
       
        ## Prune overall vocabulary
        print("sorting....")
        self.vocab = {k:v for k,v in self.vocab.items() if v >=min_freq}
        self.vocab = sorted(list(self.vocab.items()),key= lambda x: -x[1]) 
        self.vocab = [x[0] for x in self.vocab[200:]]

        # Helper variables
        self.vocab_index = dict(zip(self.vocab, range(len(self.vocab))))
        dtm_indices = [(i,self.vocab_index[w],1) for i, s in tqdm(enumerate(self.term_frequencies)) for j,(w,f) in enumerate(s.items()) if w in self.vocab_index]

        # Generate a sparse coo_matrix from non-zero coordinates. Then cast to csr_matrix. csr_matrix lets you
        # access the array just like a numpy array.
        self.dtm = coo_matrix(
            ([x[2] for x in dtm_indices], ([x[0] for x in dtm_indices],[x[1] for x in dtm_indices])), 
            shape = (len(self.term_frequencies), len(self.vocab))
        ).tocsr()
        self.cocounts = coo_matrix(da.dtm.T).dot(coo_matrix(da.dtm)) # term-term matrix
        self.cocounts.setdiag(0)
        
    def get_counts(self, word):  
        """Get the necessary counts in a unified manner"""
        i = da.vocab_index[word]
        k = da.cocounts.shape[0]
        ki = da.dtm[:, i].sum()
        kj = np.array(da.dtm.sum(0))[:,0]
        kij = da.cocounts[i].toarray()
        return k,ki,kj,kij
        
    def _dice(self, k,ki,kj,kij):
        """dice coefficient"""
        dice =  ((2*kij / (ki + kj)))[0]
        return dice
        
    def _mi(self, k,ki,kj,kij):
        """mutual information"""
        return np.log(k * kij / (ki * kj + 1e-8))[0]
  
    def get_coocs(self, word, method="dice"):
        """A function to return the list of coocurrences"""
        c = self.get_counts(word)
        if method=="dice":
            scores = self._dice(*c)
        elif method=="mi":
            scores = self._mi(*c)
        else:
            print("Unknown method !")
        sort_ind = scores.argsort()[::-1]
        return [da.vocab[i] for i in sort_ind], [scores[i] for i in sort_ind] 
    
    def semantic_network(self, word, method="dice", n=10):
        """A function to return the secondary cooccurrences"""
        neighbours = []
        coocs = self.get_coocs(word, method=method)[0][:n]
        for c in coocs:
            neighbours.append((word, c))
        for w in coocs:
            secondary_coocs = self.get_coocs(w, method=method)[0][:n]
            for c in secondary_coocs:
                neighbours.append((w, c))
        return neighbours
        

In [ ]:
da = Coocs()
da.create_dtm(content["keyword/sport_50k.txt"], min_freq=20)
#da.create_dtm(sotu, min_freq=5)

In [ ]:
da.get_coocs("Fussball", method="dice")

In [ ]:
da.semantic_network("Fussball")

In [ ]:
def visualize_semantic_network(edges, target):
    g = nx.Graph()
    g.add_edges_from(edges)
    plt.figure(3,figsize=(12,12)) 
    nx.draw(g,
             pos=nx.drawing.layout.spring_layout(g),
             with_labels=True,alpha=0.9, 
            style="dotted",
             node_color=["#1171b1" if x != word else "#ff0000" for x in g.nodes ],
            edge_color = "gray",
            font_size=12,
           node_size=40)
    plt.show()

In [ ]:
word = "Fussball"
visualize_semantic_network(da.semantic_network(word, method="dice", n=15), word)